In [1]:
!pip install pyroomacoustics
!pip install pydub
!pip install librosa
!pip install mir_eval

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.1 MB 2.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached Cython-0.29.21-cp38-cp38-manylinux1_x86_64.whl (1.9 MB)
  Using cached pybind11-2.5.0-py2.py3-none-any.whl (296 kB)
  Created wheel for pyroomacoustics: filename=pyroomacoustics-0.4.2-cp38-cp38-linux_x86_64.whl size=11577603 sha256=0f52ab17ff472cfccb91d820c03566e1b1b17a6391d622676c88040bf220bebc
  Stored in directory: /home/yongcah/.cache/pip/wheels/d7/52/4a/445bd362a578fdc26dab124aa1005598ad84562fb51acf7a46
Successfully built pyroomacoustics
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 183 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.

In [1]:
#load packages
!pip install pandas
!pip install numpy
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import fftconvolve
import IPython
import pyroomacoustics as pra
from scipy.io.wavfile import write
import librosa
from mir_eval.separation import bss_eval_sources
import pandas as pd
import math

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [36]:
output =pd.DataFrame(columns=('Namecode','Type','status'
                              ,'speed', 'starting_loc','Moving_Ang','M_Distance','M_Loc','X1','Y1','X2','Y2','X3','Y3','Length'))


s=2.7 # m/s #6.2mph or 10km/h=2.7m/s
l='(300,410)'
d=0.215 #m
p='(301, 413); (301.215, 413)' 

Type='excavator'
status='moving'
speed=s
starting_loc=l 
M_Distance=d
M_loc=p

In [37]:
#moving equipment
fs=16000
stept =200 # ms
stepn = int(stept*fs/1000)
stepd = s*stept/1000

In [38]:
output

,Namecode,Type,status,speed,starting_loc,Moving_Ang,M_Distance,M_Loc,X1,Y1,X2,Y2,X3,Y3,Length


In [39]:
def Y(theta):
    y=math.sin(math.radians(int(theta)))
    return y
def X(theta):
    x=math.cos(math.radians(int(theta)))
    return x

In [40]:
import os
def all_path(dirname):
    result = []
    for maindir, subdir, file_name_list in os.walk(dirname):
        for filename in file_name_list:
            apath = os.path.join(maindir, filename)
            result.append(apath)
    return result

In [41]:
files=all_path("./tty")

b='./output/'#output is your file to save results
d='.wav'
theta=['0','30','45','60','90','120','135','150','180','210','225','240','270','300','315','330']

for file in files: 
    fs, eq_wav = wavfile.read(file)
    movemix= np.array([])
    n = int(len(eq_wav)/stepn)
    for g in range(len(theta)):
        
        for i in range(0,n):
            room = pra.ShoeBox([400,500], fs=fs, max_order=12, absorption=0.35)  
            room.add_source([300.+X(theta[g])*stepd*i, 410.+Y(theta[g])*stepd*i], delay=0., signal=eq_wav[i*stepn:(i+1)*stepn])
            R = np.array([[301., 301.215], [413., 413.]])  
            room.add_microphone(R)
            room.simulate()
            record = room.mic_array.signals[:, 0:stepn].astype('int16')  
            if (i == 0):
                movemix =record
            else: 
                movemix = np.concatenate((movemix,record),axis=1)
            if (i+1)*stept/1000 == 1: #n*stept is the length of time for audio. Here n*stept==3,that's why the output and input are 3 seconds
                x1=300.+X(theta[g])*stepd*i
                y1=410.+Y(theta[g])*stepd*i
            elif (i+1)*stept/1000 == 2:
                x2=300.+X(theta[g])*stepd*i
                y2=410.+Y(theta[g])*stepd*i
            elif (i+1)*stept/1000 == 3:
                x3=300.+X(theta[g])*stepd*i
                y3=410.+Y(theta[g])*stepd*i 
        a=file[6:23][0:5]+file[6:23][10:17]+'_' #this step is to create the audio name,[6:23]is adjustable
        c="%s%s"%(a,g)
        pathname1=(''.join([b,c]))
        pathname=(''.join([pathname1,d]))#create pathname
        write(pathname, fs, movemix.transpose())
        output=output.append({'Namecode':c, 'Type':Type, 'status': status, 
                           'speed':s, 'starting_loc':l,'Moving_Ang':theta[g],'M_Distance':M_Distance, 'M_Loc':p ,'Length':3,'X1':x1, 'Y1':y1, 'X2':x2, 'Y2':y2,'X3':x3,'Y3':y3},ignore_index=True)

/home/yongcah/.local/lib/python3.8/site-packages/pyroomacoustics/room.py:904: UserWarning: The number of rays used for ray tracing is larger than100000 which may result in slow simulation.  The numberof rays was automatically chosen to provide accurateroom impulse response based on the room volume and thereceiver radius around the microphones.  The number ofrays may be reduced by increasing the size of thereceiver.  This tends to happen especially for largerooms with small receivers.  The receiver is a spherearound the microphone and its radius (in meters) may bespecified by providing the `receiver_radius` keywordargument to the `set_ray_tracing` method.
  warnings.warn(


In [11]:
output.to_csv('/Users/wongwingtsan/Desktop/readfile/audio.csv')#the file you wanna save the table

In [42]:
output

,Namecode,Type,status,speed,starting_loc,Moving_Ang,M_Distance,M_Loc,X1,Y1,X2,Y2,X3,Y3,Length
0,C1_02chunk_1_0,excavator,moving,2.7,"(300,410)",0,0.215,"(301, 413); (301.215, 413)",302.160000,410.000000,304.860000,410.000000,307.560000,410.000000,3
1,C1_02chunk_1_1,excavator,moving,2.7,"(300,410)",30,0.215,"(301, 413); (301.215, 413)",301.870615,411.080000,304.208883,412.430000,306.547152,413.780000,3
2,C1_02chunk_1_2,excavator,moving,2.7,"(300,410)",45,0.215,"(301, 413); (301.215, 413)",301.527351,411.527351,303.436539,413.436539,305.345727,415.345727,3
3,C1_02chunk_1_3,excavator,moving,2.7,"(300,410)",60,0.215,"(301, 413); (301.215, 413)",301.080000,411.870615,302.430000,414.208883,303.780000,416.547152,3
4,C1_02chunk_1_4,excavator,moving,2.7,"(300,410)",90,0.215,"(301, 413); (301.215, 413)",300.000000,412.160000,300.000000,414.860000,300.000000,417.560000,3
5,C1_02chunk_1_5,excavator,moving,2.7,"(300,410)",120,0.215,"(301, 413); (301.215, 413)",298.920000,411.870615,297.570000,414.208883,296.220000,416.547152,3
6,C1_02chunk_1_6,excavator,moving,2.7,"(300,410)",135,0.215,"(301, 413); (301.215, 413)",298.472649,411.527351,296.563461,413.436539,294.654273,415.345727,3
7,C1_02chunk_1_7,excavator,moving,2.7,"(300,410)",150,0.215,"(301, 413); (301.215, 413)",298.129385,411.080000,295.791117,412.430000,293.452848,413.780000,3
8,C1_02chunk_1_8,excavator,moving,2.7,"(300,410)",180,0.215,"(301, 413); (301.215, 413)",297.840000,410.000000,295.140000,410.000000,292.440000,410.000000,3
9,C1_02chunk_1_9,excavator,moving,2.7,"(300,410)",210,0.215,"(301, 413); (301.215, 413)",298.129385,408.920000,295.791117,407.570000,293.452848,406.220000,3
